# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

T. Henning  ->  T. Henning  |  ['T. Henning']
J. Li  ->  J. Li  |  ['J. Li']
J. Li  ->  J. Li  |  ['J. Li']
J. Li  ->  J. Li  |  ['J. Li']


M. Schirmer  ->  M. Schirmer  |  ['M. Schirmer']
R. Burn  ->  R. Burn  |  ['R. Burn']
Arxiv has 80 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2502.05359


extracting tarball to tmp_2502.05359...

 done.


T. Henning  ->  T. Henning  |  ['T. Henning']
Retrieving document from  https://arxiv.org/e-print/2502.05480


...

Found 89 bibliographic references in tmp_2502.06505/ms.bbl.
Retrieving document from  https://arxiv.org/e-print/2502.06553


extracting tarball to tmp_2502.06553...

 done.


R. Burn  ->  R. Burn  |  ['R. Burn']


Found 85 bibliographic references in tmp_2502.06553/article.bbl.
Issues with the citations
syntax error in line 4: '=' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.06505-b31b1b.svg)](https://arxiv.org/abs/2502.06505) | **Euclid: A complete Einstein ring in NGC 6505**  |
|| C. M. O'Riordan, et al. -- incl., <mark>M. Schirmer</mark> |
|*Appeared on*| *2025-02-11*|
|*Comments*| *Accepted in A&A. Press release: this https URL*|
|**Abstract**|            We report the discovery of a complete Einstein ring around the elliptical galaxy NGC 6505, at $z=0.042$. This is the first strong gravitational lens discovered in Euclid and the first in an NGC object from any survey. The combination of the low redshift of the lens galaxy, the brightness of the source galaxy ($I_\mathrm{E}=18.1$ lensed, $I_\mathrm{E}=21.3$ unlensed), and the completeness of the ring make this an exceptionally rare strong lens, unidentified until its observation by Euclid. We present deep imaging data of the lens from the Euclid Visible Camera (VIS) and Near-Infrared Spectrometer and Photometer (NISP) instruments, as well as resolved spectroscopy from the Keck Cosmic Web Imager (KCWI). The Euclid imaging in particular presents one of the highest signal-to-noise ratio optical/near-infrared observations of a strong gravitational lens to date. From the KCWI data we measure a source redshift of $z=0.406$. Using data from the Dark Energy Spectroscopic Instrument (DESI) we measure a velocity dispersion for the lens galaxy of $\sigma_\star=303\pm15\,\mathrm{kms}^{-1}$. We model the lens galaxy light in detail, revealing angular structure that varies inside the Einstein ring. After subtracting this light model from the VIS observation, we model the strongly lensed images, finding an Einstein radius of 2.5 arcsec, corresponding to $2.1\,\mathrm{kpc}$ at the redshift of the lens. This is small compared to the effective radius of the galaxy, $R_\mathrm{eff}\sim 12.3\,\mathrm{arcsec}$. Combining the strong lensing measurements with analysis of the spectroscopic data we estimate a dark matter fraction inside the Einstein radius of $f_\mathrm{DM} = (11.1_{-3.5}^{+5.4})\%$ and a stellar initial mass-function (IMF) mismatch parameter of $\alpha_\mathrm{IMF} = 1.26_{-0.08}^{+0.05}$, indicating a heavier-than-Chabrier IMF in the centre of the galaxy.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.06553-b31b1b.svg)](https://arxiv.org/abs/2502.06553) | **Radial velocity homogeneous analysis of M dwarfs observed with HARPS. II. Detection limits and planetary occurrence statistics**  |
|| L. Mignon, et al. -- incl., <mark>R. Burn</mark> |
|*Appeared on*| *2025-02-11*|
|*Comments*| **|
|**Abstract**|            We re-determine planetary occurrences around M dwarfs using 20 years of observations from HARPS on 197 targets. The first aim of this study is to propose more precise occurrence rates using the large volume of the sample but also variations to previous calculations, particularly by considering multiplicity, which is now an integral part of planetary occurrence calculations. The second aim is to exploit the extreme longevity of HARPS to determine occurrence rates in the unexplored domain of very long periods. This work relies entirely on the 197 radial velocity time series obtained and analysed in our previous study. By considering they are cleaned of any detectable signal, we convert them into detection limits. We use these 197 limits to produce a detectability map and combine it with confirmed planet detections to establish our occurrence rates. Finally, we also convert the detection limits from orbital period to insolation in order to construct an occurrence statistics for the temperate zone. We find a strong prevalence of low-mass planets around M dwarfs, with an occurrence rate of 120% for planets with a mass between 0.75 and 3 Me. In addition, we compute an occurrence rate of 45.3% +20-16% for temperate zone planets around M dwarfs. We obtain an occurrence rate of a few percent for giant planets with wide separations. In our sample these giant planets with wide separations are only detected around the most massive M dwarfs.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.05480-b31b1b.svg)](https://arxiv.org/abs/2502.05480) | **Finding Quasars Behind the Galactic Plane: Spectroscopic Identifications of ~1300 New Quasars at |b|<=20 degree from LAMOST DR10**  |
|| Z.-Y. Huo, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-02-11*|
|*Comments*| *15 pages, 7 figures and 2 tables, accepted for publication in ApJS*|
|**Abstract**|            Quasars behind the Galactic plane (GPQs) are excellent tracers to probe the chemistry and kinematics of the interstellar/intergalactic medium (ISM/IGM) of the Milky Way along sight lines via absorption line spectroscopy. Moreover, the quasars located at low Galactic latitudes will fill the gap in the spatial distribution of known quasars near the Galactic plane, and can be used to construct an astrometric reference frame for accurate measurements of proper motions (PMs) of stars, and substructures of the Milky Way. We started a survey of background quasars in the low Galactic latitude region since the LAMOST phase II survey in 2017. Quasar candidates have been selected from the optical and infrared photometric data of Pan-STARRS1 and WISE surveys based on their variability and color properties. In this paper, we present a sample of 1982 spectroscopically confirmed GPQs with |b| <= 20 degree based on LAMOST Data Release 10 (DR10). Among them, 1338 are newly discovered. Most GPQs are located around 240<l<90 degree, and the spatial distributions are non-uniform. These GPQs have a magnitude distribution with a peak at i-mag 19.0, and mostly around 18.0-19.5mag. The peak of redshift distributions is around ~1.5, and most GPQs have redshifts between 0.3 and 2.5. Our finding demonstrates the potential discovery space for the GPQs from the spectroscopic surveys and the promising applications for future research.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.05657-b31b1b.svg)](https://arxiv.org/abs/2502.05657) | **Ideas and Requirements for the Global Cosmic-Ray Observatory (GCOS)**  |
|| M. Ahlers, et al. |
|*Appeared on*| *2025-02-11*|
|*Comments*| *48 pages, 27 figures*|
|**Abstract**|            After a successful kick-off meeting in 2021. two workshops in 2022 and 2023 on the future Global Cosmic-Ray Observatory (GCOS) focused mainly on a straw man design of the detector and science possibilities for astro- and particle physics. About 100 participants gathered for in-person and hybrid panel discussions. In this report, we summarize these discussions, present a preliminary straw-man design for GCOS and collect short write-ups of the flash talks given during the focus sessions.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.05805-b31b1b.svg)](https://arxiv.org/abs/2502.05805) | **The SUPERCOLD-CGM survey: II. [\ion{C}{1}]$(1-0)$ emission and the physical conditions of cold gas in Enormous Ly$\alpha$ nebulae at $z\,\sim\,2$**  |
|| <mark>J. Li</mark>, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-02-11*|
|*Comments*| *27 pages, 9 figures*|
|**Abstract**|            We report ALMA and ACA observations of atomic carbon ([\ion{C}{1}]$(1-0)$) and dust continuum in 10 Enormous Ly$\alpha$ Nebulae hosting ultra-luminous Type-I QSOs at $z=2.2-2.5$, as part of the SUrvey of Protocluster ELANe Revealing CO/CI in the Ly$\alpha$ Detected CGM (SUPERCOLD-CGM). We detect [\ion{C}{1}]$(1-0)$ and dust in all ten QSOs and five companion galaxies. We find that the QSOs and companions have higher gas densities and more intense radiation fields than Luminous Infrared galaxies and high-$z$ main sequence galaxies, with the highest values found in the QSOs. By comparing molecular gas masses derived from [\ion{C}{1}]$(1-0)$, CO(4$-$3) and dust continuum, we find that the QSOs and companions display a similar low CO conversion factor of $\alpha_{\rm CO}$\,$\sim$\,0.8 $\rm M_{\sun}$${[\rm K\,km/s\,pc^2]}^{-1}$. After tapering our data to low resolution, the [\ion{C}{1}]$(1-0)$ flux increases for nine QSOs, hinting at the possibility of [\ion{C}{1}]$(1-0)$ in the circum-galactic medium (CGM) on a scale of 16$-$40 kpc. However, the [\ion{C}{1}]$(1-0)$ sensitivity is too low to confirm this for individual targets, except for a tentative (2.7$\sigma$) CGM detection in Q0050+0051{} with M$_{\rm H_2}$\,=\, ($1.0 - 2.8$)$\times 10^{10}$ $\rm M_{\sun}$. The 3$\sigma$ mass limits of molecular CGM for the remaining QSO fields are ($0.2-1.4$)\,$\times$\,10$^{10}$ $\rm M_{\sun}$. This translates into a baryon fraction of $<$0.4-3$\% $ in the molecular CGM relative to the total baryonic halo mass. Our sample also includes a radio-detected AGN, Q1416+2649{}, which shows [\ion{C}{1}]$(1-0)$ and CO(4$-$3) luminosities an order of magnitude fainter for its far-infrared luminosity than other QSOs in our sample, possibly due to a lower molecular gas mass.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.05359-b31b1b.svg)](https://arxiv.org/abs/2502.05359) | **On the Orbit of the Binary Brown Dwarf Companion GL229 Ba and Bb**  |
|| W. Thompson, et al. -- incl., <mark>T. Henning</mark> |
|*Appeared on*| *2025-02-11*|
|*Comments*| *Resubmitted to AJ*|
|**Abstract**|            The companion GL229B was recently resolved by Xuan et al. (2024) as a tight binary of two brown dwarfs (Ba and Bb) through VLTI-GRAVITY interferometry and VLT-CRIRES+ RV measurements. Here, we present Bayesian models of the interferometric and RV data in additional detail, along with an updated outer orbit of the brown dwarf pair about the primary. To create a model of the inner orbit with robust uncertainties, we apply kernel phases to the GRAVITY data to address baseline redundancy in the raw closure phases. Using parallel tempering, we constrain the binary's orbit using only VLTI-GRAVITY data, despite each epoch having low visibility-plane coverage and/or SNR. We demonstrate very agreement the VLTI-GRAVITY and CRIRES+ datasets and find that the inner binary has a period of 12.1346$\pm$0.0011 days, eccentricity of 0.2317$\pm$0.0025, and total mass of 71.0$\pm$0.4 Mjup, with Ba and Bb having masses of 37.7$\pm$1.1Mjup and 33.4$\pm$1.0Mjup respectively. With new Keck/NIRC2 astrometry, we update the outer orbit GL229B around the primary. We find a semi-major axis of 42.9+3.0-2.4AU, eccentricity of 0.736$\pm$0.014, and a total mass for B of 71.7$\pm$0.6Mjup, consistent with that derived from the inner orbit. We find a mutual inclination of 31$\pm$2.5deg, below the threshold for Kozai-Lidov oscillations. The agreement on the mass of Ba+Bb between the inner and outer orbits is an important test of our ability to model RV, astrometry, and Hipparcos-Gaia proper motion anomaly. Our methodological advances in handling interferometric data with low SNR and sparse UV-coverage will benefit future observations of rapidly-orbiting companions with VLTI-GRAVITY.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error bad escape \e at position 35</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2502.06505/./isophote_properties_new_m1.png', 'tmp_2502.06505/./light_model.png', 'tmp_2502.06505/./lens_model.png']
copying  tmp_2502.06505/./isophote_properties_new_m1.png to _build/html/
copying  tmp_2502.06505/./light_model.png to _build/html/
copying  tmp_2502.06505/./lens_model.png to _build/html/
exported in  _build/html/2502.06505.md
    + _build/html/tmp_2502.06505/./isophote_properties_new_m1.png
    + _build/html/tmp_2502.06505/./light_model.png
    + _build/html/tmp_2502.06505/./lens_model.png
found figures ['tmp_2502.06553/./stat-occu-pla_cand_sup35_decale.png', 'tmp_2502.06553/./stat-occu-pla_cand_inf35_decale.png', 'tmp_2502.06553/./stat-occu_cand_gde.png']
copying  tmp_2502.06553/./stat-occu-pla_cand_sup35_decale.png to _build/html/
copying  tmp_2502.06553/./stat-occu-pla_cand_inf35_decale.png to _build/html/
copying  tmp_2502.06553/./stat-occu_cand_gde.png to _build/html/
exported in  _build/html/2502.06553.md
    + _build/html/tmp_2502.06553/./stat

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\zsvalue}{0.4058}$
$\newcommand{\lensname}{Altieri's lens}$
$\newcommand{\galaxyname}{NGC~6505}$
$\newcommand{\galaxycoords}{RA~\ra{17;51;07.46}, Dec~+\ang{65;31}\ang{;;50.78}}$
$\newcommand{\zl}{z_\mathrm{d}}$
$\newcommand{\zs}{z_\mathrm{s}}$
$\newcommand{\erad}{\theta_\mathrm{E}}$
$\newcommand{\sigmav}{\sigma_\mathrm{v}}$
$\newcommand{\dmfraction}{f_\mathrm{DM}}$
$\newcommand{\Dls}{D_\mathrm{ls}}$
$\newcommand{\Dl}{D_\mathrm{l}}$
$\newcommand{\etheta}{\theta_q}$
$\newcommand{\lensx}{\theta_{\mathrm{l}_x}}$
$\newcommand{\lensy}{\theta_{\mathrm{l}_y}}$
$\newcommand{\posa}{\phi_\mathrm{L}}$
$\newcommand{\lambdas}{\lambda_\mathrm{s}}$
$\newcommand{\extshear}{\gamma_\mathrm{ext}}$
$\newcommand{\imfmismatch}{\alpha_\mathrm{IMF}}$
$\newcommand{\Reff}{R_\mathrm{eff}}$
$\newcommand{\chisqnu}{\chi^2_\nu}$
$\newcommand{\ndof}{N_\mathrm{dof}}$
$\newcommand$
$\newcommand{\prob}[2]{\mathrm{Pr}\left(#1\vert#2\right)}$
$\newcommand{\prior}[1]{\mathrm{Pr}\left(#1\right)}$
$\newcommand{\ev}{\varepsilon}$
$\newcommand{\Msun}{M_\odot}$
$\newcommand{\orcid}[1]$
$\newcommand{\mstarChab}{M_{\star}^\mathrm{Chab}}$
$\newcommand{\mstarmod}{M_{\star}^\mathrm{mod}}$</div>



<div id="title">

#     $\Euclid$\/: A complete Einstein ring in NGC 6505    $\thanks{This paper is published on behalf of the Euclid Consortium}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2502.06505-b31b1b.svg)](https://arxiv.org/abs/2502.06505)<mark>Appeared on: 2025-02-11</mark> -  _Accepted in A&A. Press release: this https URL_

</div>
<div id="authors">

C. M. O'Riordan, et al. -- incl., <mark>M. Schirmer</mark>

</div>
<div id="abstract">

**Abstract:** We report the discovery of a complete Einstein ring around the elliptical galaxy $\galaxyname$ , at $z=0.042$ . This is the first strong gravitational lens discovered in $\Euclid$ and the first in an NGC object from any survey. The combination of the low redshift of the lens galaxy, the brightness of the source galaxy ( $\IE=18.1$ lensed, $\IE=21.3$ unlensed), and the completeness of the ring make this an exceptionally rare strong lens, unidentified until its observation by $\Euclid$ . We present deep imaging data of the lens from the $\Euclid$ Visible Camera (VIS) and Near-Infrared Spectrometer and Photometer (NISP) instruments, as well as resolved spectroscopy from the _Keck_ Cosmic Web Imager (KCWI). The $\Euclid$ imaging in particular presents one of the highest signal-to-noise ratio optical/near-infrared observations of a strong gravitational lens to date. From the KCWI data we measure a source redshift of $z=0.406$ . Using data from the Dark Energy Spectroscopic Instrument (DESI) we measure a velocity dispersion for the lens galaxy of $\sigma_\star=303\pm15 \kms$ . We model the lens galaxy light in detail, revealing angular structure that varies inside the Einstein ring. After subtracting this light model from the VIS observation, we model the strongly lensed images, finding an Einstein radius of $\ang{;;2.5}$ , corresponding to $2.1 \mathrm{kpc}$ at the redshift of the lens. This is small compared to the effective radius of the galaxy, $R_\mathrm{eff}\sim \ang{;;12.3}$ . Combining the strong lensing measurements with analysis of the spectroscopic data we estimate a dark matter fraction inside the Einstein radius of $\dmfraction = (11.1_{-3.5}^{+5.4})\%$ and a stellar initial mass-function (IMF) mismatch parameter of $\imfmismatch = 1.26_{-0.08}^{+0.05}$ , indicating a heavier-than-Chabrier IMF in the centre of the galaxy.

</div>

<div id="div_fig1">

<img src="tmp_2502.06505/./isophote_properties_new_m1.png" alt="Fig1" width="100%"/>

**Figure 1. -** Properties of the lens galaxy isophotes as a function of the elliptical semi-major axis. _Top panel_: the axis ratio, $q=b/a$, _middle panel_: the position angle of the major axis, _bottom panel_: the total amplitude of multipole perturbations of orders $m=1,3,4$. The multipole perturbations are formulated such that a positive order $m=4$ amplitude produces boxiness. The dotted horizontal line and shaded area indicate the median $m=4$ amplitude measured by the lens modelling only, and its uncertainty respectively. The vertical dashed line indicates the elliptical radius at the critical curve, equivalent to the Einstein radius. (*fig:isophote-properties*)

</div>
<div id="div_fig2">

<img src="tmp_2502.06505/./light_model.png" alt="Fig7" width="100%"/>

**Figure 7. -** Modelling of the lens galaxy light profile. _Left_: Isophotes from the final light model plotted with data isophotes in the inner region of NGC 6505. The lensed source emission and emission from other compact sources are visible in the data isophotes. _Middle_: The difference between the data and the model, normalised by the noise in each pixel. Empty pixels are those masked from the fitting procedure. The residuals here show no evidence of a central strongly lensed image. _Right_: The lens light subtracted data, to be used for strong lens modelling presented in Sect. \ref{sec:lens-modelling}. The white circle indicates the compact source of emission included in the lens model. (*fig:vis-light-model*)

</div>
<div id="div_fig3">

<img src="tmp_2502.06505/./lens_model.png" alt="Fig8" width="100%"/>

**Figure 8. -** Elliptical power-law plus external shear and multipole model of the strongly lensed images. (A) the VIS data with the four lensed images labelled as in Table \ref{tab:image-properties}, (B) the maximum a posteriori model, both in arbitrary flux units, (C) the normalised residuals, and (D) the pixellated source reconstruction in the same units as the data and model. Solid curves in the image plane and source plane are the critical curves and caustics respectively. In (A) and (B) the mask used to model the data is indicated by the dotted white shapes. The physical scale in the source plane is indicated. (*fig:lens-modelling*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2502.06505"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\gcnote}[1]{\textcolor{orange}{\textbf{[GC: #1]}}}$
$\newcommand{\thefootnote}{\arabic{footnote}}$
$\newcommand{\footnoterule}$
$\newcommand{\footnoterule}$
$\newcommand{\footnoterule}$
$\newcommand\cl{#1}$
$\newcommand\cf{#1}$
$\newcommand\cn{#1}$
$\newcommand\cx{#1}$
$\newcommand\rb{#1}$</div>



<div id="title">

# Radial velocity homogeneous analysis of M dwarfs observed with HARPS. II. Detection limits and planetary occurrence statistics

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2502.06553-b31b1b.svg)](https://arxiv.org/abs/2502.06553)<mark>Appeared on: 2025-02-11</mark> - 

</div>
<div id="authors">

L. Mignon, et al. -- incl., <mark>R. Burn</mark>

</div>
<div id="abstract">

**Abstract:**            We re-determine planetary occurrences around M dwarfs using 20 years of observations from HARPS on 197 targets. The first aim of this study is to propose more precise occurrence rates using the large volume of the sample but also variations to previous calculations, particularly by considering multiplicity, which is now an integral part of planetary occurrence calculations. The second aim is to exploit the extreme longevity of HARPS to determine occurrence rates in the unexplored domain of very long periods. This work relies entirely on the 197 radial velocity time series obtained and analysed in our previous study. By considering they are cleaned of any detectable signal, we convert them into detection limits. We use these 197 limits to produce a detectability map and combine it with confirmed planet detections to establish our occurrence rates. Finally, we also convert the detection limits from orbital period to insolation in order to construct an occurrence statistics for the temperate zone. We find a strong prevalence of low-mass planets around M dwarfs, with an occurrence rate of 120% for planets with a mass between 0.75 and 3 Me. In addition, we compute an occurrence rate of 45.3% +20-16% for temperate zone planets around M dwarfs. We obtain an occurrence rate of a few percent for giant planets with wide separations. In our sample these giant planets with wide separations are only detected around the most massive M dwarfs.         

</div>

<div id="div_fig1">

<img src="tmp_2502.06553/./stat-occu-pla_cand_sup35_decale.png" alt="Fig3" width="100%"/>

**Figure 3. -** Planetary occurrence statistics obtained on the sub-sample of M-dwarfs with $\rm M_* > 0.35 M_{\odot}$ (*statocc_sup35*)

</div>
<div id="div_fig2">

<img src="tmp_2502.06553/./stat-occu-pla_cand_inf35_decale.png" alt="Fig4" width="100%"/>

**Figure 4. -** Planetary occurrence statistics obtained on the sub-sample of M-dwarfs with $\rm M_* < 0.35 M_{\odot}$ (*statocc_inf35*)

</div>
<div id="div_fig3">

<img src="tmp_2502.06553/./stat-occu_cand_gde.png" alt="Fig11" width="100%"/>

**Figure 11. -** Planetary occurrence statistic. Colours in the background refer to the detection limit map, from the non-reachable region (grey) to the unmissable one (white). The grey points represent the considered detections while the red points represent the candidates. The grey horizontal and vertical lines delineate the probed domains. The computed occurrence rates (along with their corresponding statistical bounds) are indicated at the center of each domain. (*statocc*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2502.06553"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

329  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

5  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
